# Guided Project: Building a database for crime reports

The goal of this guided project is for you to create a database named crimes_db with a table – boston_crimes – with appropriate datatypes for storing the data from the boston.csv file.

You will be creating the table inside a schema named crimes. 

You will also create the readonly and readwrite groups with the appropriate privileges as we have discussed in this course. 

Finally, you will also need to create one user for each of these groups.

## Create Database

In [3]:
import psycopg2

conn = psycopg2.connect(dbname = 'dq', user = 'dq')
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE DATABASE crime_db;")
conn.autocommit = False
conn.close()

DuplicateDatabase: database "crime_db" already exists


## Create Schema

In [5]:
conn = psycopg2.connect(dbname = 'crime_db', user='dq')
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes")
conn.autocommit = False

DuplicateSchema: schema "crimes" already exists


## Read file & Determine data type

In [6]:
import csv
with open("boston.csv") as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)
print(col_headers)
print(first_row)
    

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [14]:
def get_col_set(csv_filename, col_index):
    import csv
    result = set()
    with open(csv_filename) as f:
        next(f)
        reader = csv.reader(f)        
        for row in reader:
            result.add(row[col_index])
    return result
for i in range(len(col_headers)):
    values = get_col_set("boston.csv",i)
    print(col_headers[i], len(values), sep='\t')

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


## Finding the Maximum Length

In [20]:
print(col_headers)
descriptions = get_col_set("boston.csv",2)
max_len = 0
for description in descriptions:
    max_len = max(max_len, len(description))
print(max_len)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
58


In [21]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


## Create Table

* Total 7 columns
1. incident_number integer Primary Key
2. offense_code integer
3. description string --> varchar (100) > 58
4. date date
5. day_of_the_week string -> ENUM_TYPE
6. lat decimal
7. long decimal

We should create No.5 enumerated datatype with 7 string value (7 days of week first)

In [26]:
# Take care of 7 days of week as ENUM first
cur.execute("""
    CREATE TYPE daysofweek AS ENUM (
    'Monday',
    'Tuesday',
    'Wednesday',
    'Thursday',
    'Friday',
    'Saturday',
    'Sunday');
    """)

DuplicateObject: type "daysofweek" already exists


In [27]:
#Let's create table
cur.execute("""
    CREATE TABLE crimes.boston_crimes(
        incident_number integer Primary Key,
        offense_code int,
        description varchar(100),
        date DATE,
        day_of_the_week daysofweek,
        lat decimal,
        long decimal
    );
""")

## Load data into table

* using copy_expert() method

In [29]:
with open('boston.csv') as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN with CSV HEADER;",f)

UniqueViolation: duplicate key value violates unique constraint "boston_crimes_pkey"
DETAIL:  Key (incident_number)=(1) already exists.
CONTEXT:  COPY boston_crimes, line 2


In [30]:
cur.execute("SELECT * FROM crimes.boston_crimes;")
print(len(cur.fetchall()))

298329


## Revoke public privileges

In [35]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

## Create user group

In [36]:
#readonly first
cur.execute("CREATE GROUP readonly NOLOGIN")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly")
cur.execute("GRANT USAGE ON SCHEMA crimes to readonly")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes to readonly")

#read write settup
cur.execute("CREATE GROUP readwrite NOLOGIN")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite")
cur.execute("GRANT USAGE ON SCHEMA crimes to readwrite")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes to readonly")

## Create user for each group

In [37]:
#create data_analyst user
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly to data_analyst;")

#create data_scientist user
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite to data_scientist;")

## Test & Queries

In [39]:
#close the old connection to test with a brand new connection
conn.close()

conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()
# check users and groups
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
for user in cur:
    print(user)
print()
# check privileges
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
for user in cur:
    print(user)
conn.close()

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)

('readonly', 'INSERT')
('readonly', 'SELECT')
('readonly', 'UPDATE')
('readonly', 'DELETE')
